# To do

* L-BFGS の実装（多分 torch.optim を使用すれば良い）
* 綺麗な可視化
* 記事にする

In [27]:
import importlib

import torch
import numpy as np
import plotly.graph_objects as go

torch.set_default_dtype(torch.float64)

import linalg_utils
importlib.reload(linalg_utils)

import misc_utils
importlib.reload(misc_utils)

import gp_utils
importlib.reload(gp_utils)

import gp_regression
importlib.reload(gp_regression)
from gp_regression import GP_regression
from gp_regression import Sparse_GP_regression_SoR 
from gp_regression import Sparse_GP_regression_DTC
from gp_regression import Sparse_GP_regression_FITC

# 実験


## 準備

In [10]:
class Latent_Function():
    def __init__(self, noise_level=0.1):
        self.noise_level = noise_level
        self.noise = None

    def _f(self, X):
        # ここを好きにカスタマイズ
        tmp = np.sin(X) + np.cos(X)
        return tmp
    
    def f(self, X, observed=False):
        tmp = self._f(X)

        if observed is True:
            self.noise = np.random.rand(*X.shape) * self.noise_level

            if isinstance(X, np.ndarray):
                return np.array(tmp) + self.noise
            elif isinstance(X, torch.Tensor):
                return tmp + torch.Tensor(self.noise)
        
        else:
            if isinstance(X, np.ndarray):
                return np.array(tmp)
            elif isinstance(X, torch.Tensor):
                return tmp


def make_data(f, X=None, X_pred=None):
    if X is None:
        X_normal = torch.randn(50) * 1.5 + 4 
        X_normal = torch.clip(X_normal, 0, 10) 
        X_uniform = torch.rand(50) * 6 + 2  
        X_combined = torch.cat([X_normal, X_uniform])
        X, _ = torch.sort(X_combined)
        X = X.reshape(-1, 1)

    y = f(X, observed=True)

    if X_pred is None:
        X_max, X_min = X.max(), X.min()
        interval = torch.abs(X_max - X_min)
        size = int(torch.ceil(interval / 0.1))
        X_pred_max, X_pred_min = X_max + interval * 0.2, X_min - interval * 0.2
        X_pred = torch.linspace(X_pred_min, X_pred_max, size).reshape(-1, 1)

    return X, y, X_pred


def visualize(X_pred, f, mean, cov, p_inputs=None, title=""):
    mean = mean.ravel().detach().numpy()
    var = torch.diagonal(cov).detach().numpy()
    X_pred = X_pred.ravel().detach().numpy()

    if p_inputs is not None:
        p_inputs = p_inputs.ravel().detach().numpy()

    credible_interval = 1.96 * np.sqrt(var) # 95%

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_pred, y=f(X_pred), line_color="#00CC00", name="Latent Function"))
    fig.add_trace(go.Scatter(x=X_pred, y=mean, line_color="#FE73FF", name="Mean"))
    fig.add_trace(go.Scatter(x=X_pred, y=mean-credible_interval, mode='lines', line=dict(color='lightgray'), showlegend=False))
    fig.add_trace(go.Scatter(x=X_pred, y=mean+credible_interval, mode='lines', line=dict(color='lightgray'), fill='tonexty', showlegend=False))
    
    if p_inputs is not None:
        fig.add_trace(go.Scatter(x=p_inputs, y=f(p_inputs), mode='markers', marker=dict(size=8, color="#FF0000"), name="Pseudo-inputs"))

    fig.update_layout(title=title, xaxis_title="X", yaxis_title="f*")
    fig.show()

In [11]:
f = Latent_Function(noise_level=0.1).f
X, y, X_pred = make_data(f)

## GP regression

In [16]:
model1 = GP_regression(X, y)
mean1, cov1 = model1.predict(X_pred.clone())

visualize(X_pred, f, mean1, cov1, title="GP_regression")

In [17]:
model1.params

[tensor(0., requires_grad=True),
 tensor(0., requires_grad=True),
 tensor(-0.6931, requires_grad=True)]

In [18]:
model1.optimize(iteration=200, learning_rate=0.01)
mean1_opt, cov1_opt = model1.predict(X_pred.clone())

visualize(X_pred, f, mean1_opt, cov1_opt, title="Optimized GP_regression")

[tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(-0.6931, requires_grad=True)]
opt_iter: 1/200
params: [1.0, 1.0, 0.5]
grads: [tensor(98.9665), tensor(-3.1069), tensor(-2.1014)]
opt_iter: 2/200
params: [0.9900498337501684, 1.0100501670516577, 0.5050250835180512]
grads: [tensor(99.9792), tensor(-3.0731), tensor(-2.0671)]
opt_iter: 3/200
params: [0.9801961992459274, 1.0201982814913166, 0.5100983064872415]
grads: [tensor(101.0018), tensor(-3.0396), tensor(-2.0331)]
opt_iter: 4/200
params: [0.9704365594480294, 1.030443199639634, 0.5152184985970695]
grads: [tensor(102.0348), tensor(-3.0063), tensor(-1.9994)]
opt_iter: 5/200
params: [0.9607684595167064, 1.0407837062299063, 0.5203844230630615]
grads: [tensor(103.0783), tensor(-2.9733), tensor(-1.9662)]
opt_iter: 6/200
params: [0.9511895276350152, 1.05121851941201, 0.5255947794319401]
grads: [tensor(104.1327), tensor(-2.9406), tensor(-1.9332)]
opt_iter: 7/200
params: [0.9416974756221719, 1.0617462962028692, 0.53084820666

## SoR

In [259]:
model2.pseudo_inputs

[tensor(0.1353, requires_grad=True),
 tensor(0.9764, requires_grad=True),
 tensor(1.8175, requires_grad=True),
 tensor(2.6587, requires_grad=True),
 tensor(3.4998, requires_grad=True),
 tensor(4.3409, requires_grad=True),
 tensor(5.1820, requires_grad=True),
 tensor(6.0232, requires_grad=True),
 tensor(6.8643, requires_grad=True),
 tensor(7.7054, requires_grad=True)]

In [254]:
torch.Tensor(model2.pseudo_inputs)

tensor([0.1353, 0.9764, 1.8175, 2.6587, 3.4998, 4.3409, 5.1820, 6.0232, 6.8643,
        7.7054])

In [253]:
[np.float64(model2.pseudo_inputs[i]) for i in range(len(model2.pseudo_inputs))]

[0.1352783418869592,
 0.976404679745356,
 1.817531017603753,
 2.6586573554621498,
 3.4997836933205466,
 4.340910031178943,
 5.18203636903734,
 6.023162706895738,
 6.864289044754134,
 7.7054153826125305]

In [5]:
pseudo_inputs = [torch.tensor(0.1353, requires_grad=True),
                 torch.tensor(0.9764, requires_grad=True),
                 torch.tensor(1.8175, requires_grad=True),
                 torch.tensor(2.6587, requires_grad=True),
                 torch.tensor(3.4998, requires_grad=True),
                 torch.tensor(4.3409, requires_grad=True),
                 torch.tensor(5.1820, requires_grad=True),
                 torch.tensor(6.0232, requires_grad=True),
                 torch.tensor(6.8643, requires_grad=True),
                 torch.tensor(7.7054, requires_grad=True)]

In [28]:
model2 = Sparse_GP_regression_SoR(X, y, pseudo_inputs=pseudo_inputs, p_optimized=True)
mean2, cov2 = model2.predict(X_pred.clone())

visualize(X_pred, f, mean2, cov2, p_inputs=torch.Tensor(model2.pseudo_inputs), title="SoR")

In [24]:
model2.params

[tensor(0., requires_grad=True),
 tensor(0., requires_grad=True),
 tensor(-0.6931, requires_grad=True),
 tensor(0.1353, requires_grad=True),
 tensor(0.9764, requires_grad=True),
 tensor(1.8175, requires_grad=True),
 tensor(2.6587, requires_grad=True),
 tensor(3.4998, requires_grad=True),
 tensor(4.3409, requires_grad=True),
 tensor(5.1820, requires_grad=True),
 tensor(6.0232, requires_grad=True),
 tensor(6.8643, requires_grad=True),
 tensor(7.7054, requires_grad=True)]

In [26]:
torch.Tensor(pseudo_inputs).requires_grad(True)

tensor([0.1353, 0.9764, 1.8175, 2.6587, 3.4998, 4.3409, 5.1820, 6.0232, 6.8643,
        7.7054])

In [29]:
model2.optimize(iteration=1000, learning_rate=0.01)
mean2_opt, cov2_opt = model2.predict(X_pred.clone())

visualize(X_pred, f, mean2_opt, cov2_opt, p_inputs=torch.Tensor(model2.pseudo_inputs), title="Optimized SoR")

[tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(-0.6931, requires_grad=True), tensor(0.1353, requires_grad=True), tensor(0.9764, requires_grad=True), tensor(1.8175, requires_grad=True), tensor(2.6587, requires_grad=True), tensor(3.4998, requires_grad=True), tensor(4.3409, requires_grad=True), tensor(5.1820, requires_grad=True), tensor(6.0232, requires_grad=True), tensor(6.8643, requires_grad=True), tensor(7.7054, requires_grad=True)]
opt_iter: 1/1000
params: [1.0, 1.0, 0.5, 0.1353, 0.9764, 1.8175, 2.6587, 3.4998, 4.3409, 5.182, 6.0232, 6.8643, 7.7054]
grads: [tensor(52.9482), tensor(-7.1186), tensor(-2.6205), None, None, None, None, None, None, None, None, None, None]
opt_iter: 2/1000
params: [0.9900498337510379, 1.0100501670699793, 0.5050250835228116, 0.1353, 0.9764, 1.8175, 2.6587, 3.4998, 4.3409, 5.182, 6.0232, 6.8643, 7.7054]
grads: [tensor(53.9736), tensor(-7.0951), tensor(-2.4775), None, None, None, None, None, None, None, None, None, None]
opt_iter: 3/100

_LinAlgError: Matrix is not positive definite, even with jitter.

## DTC

In [85]:
model3 = Sparse_GP_regression_DTC(X, y)
mean3, cov3 = model3.predict(X_pred.clone())

visualize(X_pred, f, mean3, cov3, p_inputs=model3.pseudo_inputs, title="DTC")

In [86]:
model3.optimize(iteration=200, learning_rate=0.01)
mean3_opt, cov3_opt = model3.predict(X_pred.clone())

visualize(X_pred, f, mean3_opt, cov3_opt, p_inputs=model3.pseudo_inputs, title="Optimized DTC")

opt_iter: 1/200
params: [1.0, 1.0, 0.5]

opt_iter: 2/200
params: [0.9900498337515657, 1.0100501670698419, 0.505025083506058]

opt_iter: 3/200
params: [0.9801937129034886, 1.0202004427871223, 0.5100795200830429]

opt_iter: 4/200
params: [0.9704276104008118, 1.0304512105447503, 0.515144839380925]

opt_iter: 5/200
params: [0.9607477011067993, 1.0408028451816032, 0.5201980041648484]

opt_iter: 6/200
params: [0.9511503594374545, 1.051255719107787, 0.525210597235187]

opt_iter: 7/200
params: [0.9416321566235701, 1.0618102099132833, 0.5301480298300145]

opt_iter: 8/200
params: [0.9321898575877926, 1.0724667097048541, 0.5349689034356024]

opt_iter: 9/200
params: [0.922820417430781, 1.0832256363823278, 0.5396247290359114]

opt_iter: 10/200
params: [0.9135209775289442, 1.0940874469682602, 0.5440602783283789]

opt_iter: 11/200
params: [0.9042888612570308, 1.1050526529177138, 0.5482148742338865]

opt_iter: 12/200
params: [0.8951215693622359, 1.1161218370408135, 0.5520248681498338]

opt_iter: 13/20

## FITC

In [87]:
model4 = Sparse_GP_regression_FITC(X, y)
mean4, cov4 = model4.predict(X_pred.clone())

visualize(X_pred, f, mean4, cov4, p_inputs=model4.pseudo_inputs, title="FITC")

In [88]:
model4.optimize(iteration=200, learning_rate=0.01)
mean4_opt, cov4_opt = model4.predict(X_pred.clone())

visualize(X_pred, f, mean4_opt, cov4_opt, p_inputs=model4.pseudo_inputs, title="Optimized FITC")

opt_iter: 1/200
params: [1.0, 1.0, 0.5]

opt_iter: 2/200
params: [0.9900498337515657, 1.0100501670698419, 0.505025083506058]

opt_iter: 3/200
params: [0.9801937129034886, 1.0202004427871223, 0.5100795200830429]

opt_iter: 4/200
params: [0.9704276104008118, 1.0304512105447503, 0.515144839380925]

opt_iter: 5/200
params: [0.9607477011067993, 1.0408028451816032, 0.5201980041648484]

opt_iter: 6/200
params: [0.9511503594374545, 1.051255719107787, 0.525210597235187]

opt_iter: 7/200
params: [0.9416321566235701, 1.0618102099132833, 0.5301480298300145]

opt_iter: 8/200
params: [0.9321898575877926, 1.0724667097048541, 0.5349689034356024]

opt_iter: 9/200
params: [0.922820417430781, 1.0832256363823278, 0.5396247290359114]

opt_iter: 10/200
params: [0.9135209775289442, 1.0940874469682602, 0.5440602783283789]

opt_iter: 11/200
params: [0.9042888612570308, 1.1050526529177138, 0.5482148742338865]

opt_iter: 12/200
params: [0.8951215693622359, 1.1161218370408135, 0.5520248681498338]

opt_iter: 13/20